# Amazon Bedrock AgentCore Runtime で Strands Agent を Braintrust Observability と共に使用

## 概要

このノートブックでは、Braintrust 可観測性統合を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime にデプロイする方法を説明します。この実装は Amazon Bedrock Claude モデルを使用し、OpenTelemetry (OTEL) を通じてテレメトリデータを Braintrust に送信します。

## 主なコンポーネント

- **Strands Agents**: 組み込みテレメトリサポートを持つ LLM 駆動エージェントを構築するための Python フレームワーク
- **Amazon Bedrock AgentCore Runtime**: AWS でエージェントをホスティングおよびスケーリングするためのマネージドランタイムサービス
- **Braintrust**: LLM アプリケーションの監視とデバッグのための可観測性プラットフォーム
- **OpenTelemetry**: テレメトリデータの収集とエクスポートのための業界標準プロトコル

## アーキテクチャ

エージェントはコンテナ化され、Amazon Bedrock AgentCore Runtime にデプロイされ、呼び出し用の HTTP エンドポイントを提供します。テレメトリデータは Strands エージェントから OTEL エクスポーターを通じて Braintrust に流れ、監視とデバッグに使用されます。この実装では、適切な設定順序を確保するために遅延初期化パターンを使用しています。

## 前提条件

- Python 3.10+
- Bedrock および AgentCore 権限を持つ AWS 認証情報
- API キーを持つ Braintrust アカウント。詳細は [Braintrust ウェブサイト](https://www.braintrust.dev/)を確認してください
- Docker がローカルにインストールされていること
- us-west-2 で Amazon Bedrock Claude モデルへのアクセス

## インストール

requirements.txt ファイルから必要な依存関係をインストール:

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェント実装

エージェントファイル（`strands_claude.py`）は、Web 検索機能を持つトラベルエージェントを実装しています。主な特徴:
- 遅延初期化パターンにより、環境変数後にテレメトリが設定されることを保証

In [ ]:
%%writefile strands_claude.py
import os
import logging
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from ddgs import DDGS

logging.basicConfig(level=logging.ERROR, format="[%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.getenv("AGENT_RUNTIME_LOG_LEVEL", "INFO").upper())


@tool
def web_search(query: str) -> str:
    """
    Search the web for information using DuckDuckGo.

    Args:
        query: The search query

    Returns:
        A string containing the search results
    """
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=5)

        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No summary')}\n"
                f"   Source: {result.get('href', 'No URL')}\n"
            )

        return "\n".join(formatted_results) if formatted_results else "No results found."

    except Exception as e:
        return f"Error searching the web: {str(e)}"

# Function to initialize Bedrock model
def get_bedrock_model():
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    model_id = os.getenv("BEDROCK_MODEL_ID", "global.anthropic.claude-haiku-4-5-20251001-v1:0")

    bedrock_model = BedrockModel(
        model_id=model_id,
        region_name=region,
        temperature=0.0,
        max_tokens=1024
    )
    return bedrock_model

# Initialize the Bedrock model
bedrock_model = get_bedrock_model()

# Define the agent's system prompt
system_prompt = """あなたはリアルタイムの Web 情報にアクセスできる、パーソナライズされた旅行推奨を専門とする経験豊富なトラベルエージェントです。
あなたの役割は、Web 検索を使用して最新情報を取得し、ユーザーの好みに合った理想の目的地を見つけることです。
現在の情報、簡単な説明、実用的な旅行の詳細を含む包括的な推奨を提供してください。"""

app = BedrockAgentCoreApp()

def initialize_agent():
    """Initialize the agent with proper telemetry configuration."""

    # Initialize Strands telemetry with 3P configuration
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()
    
    # Create and cache the agent
    agent = Agent(
        model=bedrock_model,
        system_prompt=system_prompt,
        tools=[web_search]
    )
    
    return agent

@app.entrypoint
def strands_agent_bedrock(payload, context=None):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    logger.info("[%s] User input: %s", context.session_id, user_input)
    
    # Initialize agent with proper configuration
    agent = initialize_agent()
    
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます。`bedrock_agentcore_starter_toolkit` を使用してエージェントを設定する場合、デフォルトで AgentCore Observability が設定されることに注意してください。Braintrust を使用するには、以下に説明するように AgentCore Observability の設定を削除する必要があります:

<div style="text-align:left">
    <img src="../images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_braintrust_observability"

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    disable_otel=True,
)
response

## AgentCore Runtime へのデプロイ

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="../images/launch.png" width="75%"/>
</div>

In [ ]:
# Braintrust configuration
otel_endpoint = "https://api.braintrust.dev/otel"
braintrust_api_key = "<braintrust-api-key>"  # For production key should be securely stored
braintrust_project_id = "<braintrust-project-id>"
otel_auth_header = f"Authorization=Bearer {braintrust_api_key}, x-bt-parent=project_id:{braintrust_project_id}"


launch_result = agentcore_runtime.launch(
    env_vars={
        "BEDROCK_MODEL_ID": "global.anthropic.claude-haiku-4-5-20251001-v1:0", # Example model ID
        "OTEL_EXPORTER_OTLP_ENDPOINT": otel_endpoint,  # Use Braintrust OTEL endpoint
        "OTEL_EXPORTER_OTLP_HEADERS": otel_auth_header,  # Add Braintrust OTEL auth header
        "DISABLE_ADOT_OBSERVABILITY": "true",
    }
)
launch_result

## デプロイステータスの確認

呼び出す前に、Runtime が準備完了になるまで待ちます:

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)

print(f"Final status: {status}")

### AgentCore Runtime の呼び出し

最後に、ペイロードで AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="../images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "I'm planning a weekend trip to Orlando. What are the must-visit places and local food I should try?"})

In [ ]:
from IPython.display import Markdown, display
display(Markdown("".join(invoke_response['response'])))

## Braintrust でトレースを表示

トレースを表示するには:
1. https://www.braintrust.dev で Braintrust ダッシュボードにアクセス
2. プロジェクト（例: 「Strands Agent Demo」）に移動
3. 「Logs」をクリックしてテレメトリデータを表示

トレースには以下が含まれます:
- エージェント呼び出しの詳細
- ツール呼び出し（Web 検索）
- レイテンシとトークン使用量を含むモデルインタラクション
- リクエスト/レスポンスペイロードなど

## クリーンアップ（オプション）

In [ ]:
import boto3
# Delete the AgentCore Runtime and ECR repository
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

ecr_client = boto3.client(
    'ecr',
    region_name=region
)

# Delete the runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

# Delete the ECR repository
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

print("Cleanup completed")

## まとめ

Braintrust 可観測性を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime に正常にデプロイしました。この実装は以下を示しています:
- Strands エージェントと AgentCore Runtime の統合
- 適切な設定順序を確保するための遅延初期化パターン
- トレースを Braintrust に送信するための OpenTelemetry の設定
- SDK と boto3 クライアントの両方を通じた呼び出し

エージェントは、Braintrust を通じた完全な可観測性を備えたマネージドでスケーラブルな環境で実行されています。